In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import pandas as pd
import re
import os

path = '/content/drive/MyDrive'

df = pd.read_csv("/content/drive/MyDrive/bbc_data.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['labels'] = le.fit_transform(df['labels'])

In [ ]:
file4 = path + '/train.txt'
file5 = path + '/test.txt'

file4_l = path + '/train_labels.txt'
file5_l = path + '/test_labels.txt'


import os

try:
    os.remove(file4)
    os.remove(file5)
    os.remove(file4_l)
    os.remove(file5_l)
except OSError:
    pass


from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)


with open(file4, "a") as f:
    for t in train['data']:
        f.write(t + '\n')
        f.write('_____')

with open(file5, "a") as f:
    for t in test['data']:
        f.write(t + '\n')
        f.write('_____')

with open(file4_l, "a") as f:
    for t in train['labels']:
        f.write(str(t) + '\n')

with open(file5_l, "a") as f:
    for t in test['labels']:
        f.write(str(t) + '\n')

In [ ]:
def split_into_sentences(text, file):

    sents = []

    text0 = re.sub('\n', ' ', text)
    text0 = re.sub('\s$|\"', '', text0)

    text1 = re.split('\s{2,}', text0)
    sents.append(text1[0]+'.')

    text2 = ' '.join(text1[1:])

    text3 = re.split('(?<=[.!?]) +', text2)

    text3 = [t for t in text3 if t != '']

    sents += text3

    with open(file, "a") as f:
        for line in sents:
            f.write(line + '\n')

    return sents



In [ ]:
def split_into_words(sents, file):

    words = []

    for s in sents:

        s0 = re.sub('xc2xa', '', s)

        s0 = re.sub(' ', '__probel__', s0)

        s0 = re.sub('(?<=(Dr|Mr|Ms))(__probel__)(?=[A-Z])', ' ', s0)


        w = re.split('__probel__', s0)
        w = [w_i for w_i in w if w_i != '' and w_i != '-']
        words += w



    with open(file, "a") as f:
        for word in words:
            f.write(word + '\n')

    return words


In [ ]:
file = path + '/sentences.txt'
file2 = path + '/words.txt'
file3 = path+'/records.tsv'


import os

try:
    os.remove(file)
    os.remove(file2)
    os.remove(file3)
except OSError:
    pass


for i in range(len(df)):
    sents = split_into_sentences(df['data'][i], file)
    split_into_words(sents, file2)

Примеры, которые корректно обрабатываются:

1.   speaker-climbing

2.   $1300

3.   9/11

4.   26.5%

5.   $15.5m

6.   26th

7.   my.post.444.@epost.ru

8.   Ms | Mr | Dr





In [ ]:
import csv

import nltk
from nltk.stem.snowball import SnowballStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

file3 = path+'/records.tsv'


def stem_and_lem(tokens):
    snow_stemmer = SnowballStemmer(language='english')

    stem_words = []
    lem_words = []

    for w in tokens:
        x = snow_stemmer.stem(w)
        y = wnl.lemmatize(w)
        stem_words.append(x)
        lem_words.append(y)


    with open(file3, 'w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')

        need_empty_line = False
        for t, s, l in zip(tokens, stem_words, lem_words):
            if need_empty_line:
                writer.writerow("")
                need_empty_line = False


            if t[-1] in ('.', '!', '?', '...', ')', '"', ','):
                if t[-1] in ('.', '!', '?', '...'):
                    need_empty_line = True
                k = t[-1]
                writer.writerow([t[:-1], s[:-1], l[:-1]])
                writer.writerow([k, k, k])
            elif t[0] in ('(', '"'):
                k = t[0]
                writer.writerow([k, k, k])
                writer.writerow([t[1:], s[1:], l[1:]])
            else:
                writer.writerow([t, s, l])


        size = tsvfile.truncate()
        size = tsvfile.truncate(size - 2)



In [ ]:
with open(file2, "r+") as f:
    lines = f.read().splitlines()
    stem_and_lem(lines)